---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [325]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # Read in data
    towns_raw = pd.read_csv("university_towns.txt", sep='\t',header = None)
    towns = towns_raw.copy()
    
    # Clean Columns
    towns['State'] = towns.apply(lambda df: df[0].split('[',1)[0] if df[0].find('edit') > 0 else np.nan,axis = 1).ffill()
    towns['RegionName'] = towns.apply(lambda df: df[0].split('(',1)[0].rstrip() if df[0].find('edit') < 0 else np.nan,axis = 1)
    towns = towns.dropna().reset_index()
    towns = towns[['State','RegionName']]
    
    # Return list of states & regions
    return towns

get_list_of_university_towns().head(10)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [24]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3
    A recession is defined as starting with two consecutive quarters of GDP decline'''
    
    # Read in Data
    gdp_raw = pd.read_excel("gdplev.xls",skiprows = 7)
    
    # Rename columns & calculate lagged variables
    gdp = (gdp_raw.copy()
       .rename(columns = {'Unnamed: 0':'Year','Unnamed: 1':'GDP_Current_Dollars','Unnamed: 2':'GDP_2009_Dollars',
                          'Unnamed: 4':'Year_Quarter', 'Unnamed: 5': 'GDP_Current_Dollars_Quarter', 'Unnamed: 6': 'GDP_2009_Dollars_Quarter'})
       .assign(GDP_Quarter_lag = gdp_raw['Unnamed: 5'].shift(),
              GDP_diff = gdp_raw['Unnamed: 5'] - gdp_raw['Unnamed: 5'].shift(),
              GDP_diff_lag = (gdp_raw['Unnamed: 5'] - gdp_raw['Unnamed: 5'].shift()).shift(),
              Year_Quarter_lag = gdp_raw['Unnamed: 4'].shift())
      )
    
    # Define "recession" quarters & identify "recession start"
    gdp['recession'] = gdp.apply(lambda df: 1 if (df['GDP_diff'] < 0) & (df['GDP_diff_lag'] < 0) else (0 if (df['GDP_diff'] >= 0) & (df['GDP_diff_lag'] >= 0) else np.nan),axis = 1).ffill()
    gdp['recession_lag'] = gdp['recession'].shift()
    recession_start = gdp[(gdp['recession'] == 1) & (gdp['recession_lag'] == 0)].reset_index()
    
    # Return list of year-quarters when a recession began
    return recession_start['Year_Quarter_lag']

get_recession_start()

0    1949q1
1    1953q3
2    1957q4
3    2008q4
Name: Year_Quarter_lag, dtype: object

In [22]:
def get_recession_end():
    
    # Read in raw data
    gdp_raw = pd.read_excel("gdplev.xls",skiprows = 7)
    
    # Rename columns and calculate lagged variables
    gdp = (gdp_raw.copy()
       .rename(columns = {'Unnamed: 0':'Year','Unnamed: 1':'GDP_Current_Dollars','Unnamed: 2':'GDP_2009_Dollars',
                          'Unnamed: 4':'Year_Quarter', 'Unnamed: 5': 'GDP_Current_Dollars_Quarter', 'Unnamed: 6': 'GDP_2009_Dollars_Quarter'})
       .assign(GDP_Quarter_lag = gdp_raw['Unnamed: 5'].shift(),
              GDP_diff = gdp_raw['Unnamed: 5'] - gdp_raw['Unnamed: 5'].shift(),
              GDP_diff_lag = (gdp_raw['Unnamed: 5'] - gdp_raw['Unnamed: 5'].shift()).shift()
               )
      )
    
    # Define "recession" quarters & identify "recession end"
    gdp['recession'] = gdp.apply(lambda df: 1 if (df['GDP_diff'] < 0) & (df['GDP_diff_lag'] < 0) else (0 if (df['GDP_diff'] >= 0) & (df['GDP_diff_lag'] >= 0) else np.nan),axis = 1)
    gdp['recession'] = gdp['recession'].ffill()
    gdp['recession_lag'] = gdp['recession'].shift()
    recession_end = gdp[(gdp['recession'] == 0) & (gdp['recession_lag'] == 1)].reset_index()

    # Return list of year-quarters when a recession ended
    return recession_end['Year_Quarter']

get_recession_end()

0    1950q2
1    1954q3
2    1958q3
3    2009q4
Name: Year_Quarter, dtype: object

In [21]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # Read in raw data
    gdp_raw = pd.read_excel("gdplev.xls",skiprows = 7)
    
    # Renamed columns and calculated lagged variables
    gdp = (gdp_raw.copy()
           .rename(columns = {'Unnamed: 0':'Year','Unnamed: 1':'GDP_Current_Dollars','Unnamed: 2':'GDP_2009_Dollars',
                              'Unnamed: 4':'Year_Quarter', 'Unnamed: 5': 'GDP_Current_Dollars_Quarter', 'Unnamed: 6': 'GDP_2009_Dollars_Quarter'})
           .assign(GDP_Quarter_lag = gdp_raw['Unnamed: 5'].shift(),
                  GDP_diff = gdp_raw['Unnamed: 5'] - gdp_raw['Unnamed: 5'].shift(),
                  GDP_diff_lag = (gdp_raw['Unnamed: 5'] - gdp_raw['Unnamed: 5'].shift()).shift()
                   )
          )
    
    # Group all the quarters in a single recession
    gdp['recession'] = gdp.apply(lambda df: 1 if (df['GDP_diff'] < 0) & (df['GDP_diff_lag'] < 0) else (0 if (df['GDP_diff'] >- 0) & (df['GDP_diff_lag'] >= 0) else np.nan),axis = 1).ffill()
    gdp['recession_lag'] = gdp['recession'].shift()
    gdp['recession_group'] = gdp.apply(lambda df: df['Year_Quarter'] if (df['recession'] == 1) & (df['recession_lag'] == 0) else np.nan,axis=1).ffill()
    min_GDP = pd.DataFrame(gdp.groupby('recession_group').GDP_Current_Dollars_Quarter.agg(min))

    # Calculate minimum GDP per recession using group indicator
    bottom = pd.merge(gdp, min_GDP, how='inner', left_on=['recession_group'], right_index=True)
    bottom = bottom[bottom['GDP_Current_Dollars_Quarter_x'] == bottom['GDP_Current_Dollars_Quarter_y']].reset_index()
    
    # Return quarter with the worst GDP for each recession
    return bottom['Year_Quarter']

get_recession_bottom()

0    1949q4
1    1954q1
2    1958q1
3    2009q2
Name: Year_Quarter, dtype: object

In [170]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"]. (Had to add in RegionId to produce correct # of rows)
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # Read in Raw Data
    house_raw = pd.read_csv("City_Zhvi_AllHomes.csv")
    
    # Create a dataframe to identiy which months belong to which quarters
    house = house_raw.copy()
    quarter = pd.DataFrame([[col,col.split('-')[0],col.split('-')[1]] for col in house.columns if '-' in col]).rename(columns = {0:'Year_month',1:'Year',2:'Month'})
    quarter['Quarter'] = quarter.apply(lambda df: 'q1' if df['Month'] in {'01','02','03'} else ('q2' if df['Month'] in {'04','05','06'} else ('q3' if df['Month'] in {'07','08','09'} else 'q4')), axis = 1)
    
    # Transform housing data set from wide to long
    house = pd.melt(house_raw,id_vars=['RegionID','RegionName','State','Metro','CountyName','SizeRank'],var_name = 'Year_month',value_name = 'Home_Sale_Price')
    
    # Merge Quarters data set for grouping purposes
    house_quarter = pd.merge(house,quarter,how = 'left',left_on='Year_month',right_on='Year_month')
    house_quarter['Year_Quarter'] = house_quarter['Year'] + house_quarter['Quarter']
    house_quarter = house_quarter[house_quarter['Year'].astype(float)>=2000]
    
    # Calculate mean house price across each quarter & place - then make wide by Year-Quarter
    quarter_sum = house_quarter.groupby(['RegionID','State','RegionName','Year_Quarter']).Home_Sale_Price.agg(np.mean).unstack()
    quarter_sum = quarter_sum.reset_index()
    quarter_sum = quarter_sum.set_index(['State','RegionName'])
    
    # Return data set of avg house prices for every year-quarter in each region
    return quarter_sum

convert_housing_data_to_quarters().shape

(10730, 68)

In [324]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Leverage previous functions to gather relevant recession quarters
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    
    # Convert housing data into quarters and then transform from wide to long
    house = convert_housing_data_to_quarters().reset_index()
    house = pd.melt(house,id_vars=['RegionID','RegionName','State'],var_name = 'Year_Quarter',value_name = 'Home_Sale_Price')
    
    # Merge recession dates onto housing data set
    recession = pd.merge(recession_start,recession_bottom, how = 'inner',left_index = True,right_index = True)
    recession = recession.reset_index().rename(columns = {'index':'recession #'})
    house_start = pd.merge(recession[['recession #','Year_Quarter_lag']],house,how = 'inner',left_on="Year_Quarter_lag",right_on="Year_Quarter")
    house_bottom = pd.merge(recession[['recession #','Year_Quarter']],house,how = 'inner',left_on="Year_Quarter",right_on="Year_Quarter")

    # Calculate the difference in home prices for each recession
    house_recession = (pd.merge(house_start,house_bottom,how='inner',left_on=['RegionID','RegionName','State','recession #'],right_on=['RegionID','RegionName','State','recession #'])
                       .rename(columns = {'Home_Sale_Price_x':'Recession_Start_Home_Price', 'Home_Sale_Price_y': 'Recession_Bottom_Home_Price','Year_Quarter_x':'Recession_Start','Year_Quarter_y':'Recession_Bottom'})
                      )
    house_recession['Home_Diff'] = house_recession['Recession_Start_Home_Price'] - house_recession['Recession_Bottom_Home_Price']
    house_recession = house_recession.drop('Year_Quarter_lag',axis =1)
    
    # Read in list of university towns & identify them as such
    uni_towns = get_list_of_university_towns()
    uni_towns['uni'] = 1

    # Merge university towns onto housing data set
    house_recession['State_Full'] = house_recession['State'].map(states)
    uni_house = pd.merge(house_recession,uni_towns,how='outer',left_on = ['State_Full','RegionName'],right_on = ['State','RegionName'])
    uni_house['uni'] = uni_house['uni'].fillna(0)
    uni_house.head()
    
    # Split data into uni & non-uni towns
    from scipy import stats
    uni = uni_house[(uni_house['uni'] == 1)]
    non_uni = uni_house[uni_house['uni'] == 0]
    
    # Calculate means for both populations
    uni_mean = uni['Home_Diff'].mean()
    non_uni_mean = non_uni['Home_Diff'].mean()
    better = (lambda uni_mean,non_uni_mean: 'university_town' if uni_mean < non_uni_mean else 'non-university town')(uni_mean,non_uni_mean)
    
    # Run t-test
    results = stats.ttest_ind(uni['Home_Diff'].dropna(), non_uni['Home_Diff'].dropna())
    
    # Return requested tuple
    return (results.pvalue < 0.01,results.pvalue, better)

run_ttest()

(False, 0.01379780236074378, 'university_town')